In [15]:
import pandas as pd
import polars as pl
import gc
import json

In [16]:
def j2d(f):
    with open(f, "rt") as file:
        d = json.load(file)
    return d

def d2j(d, f):
    with open(f, "wt") as file:
        json.dump(d, file, indent = 2)

In [17]:
f = "../../share3/jdi13641-sup-0001-tables1-s5.xlsx"
d_mst = pd.read_excel(f, sheet_name="Supplemental Table 1", skiprows=2, dtype={"diagnosis code":str})\
    .query("`diagnosis code`==`diagnosis code`")
dr_mst = pd.read_excel(f, sheet_name="Supplemental Table 2", skiprows=2, dtype={"medicine code":str})\
    .query("`medicine code`==`medicine code`")\
    .rename(columns = {"YJ code":"yj"})\
    .assign(yj_7 = lambda df:df.yj.str[:7])
d_mst.head()
dm_icd10 = set(d_mst["ICD-10 code"].unique().tolist())
dm_codes = set(d_mst["diagnosis code"].unique().tolist())
# dm_icd10

dr_codes = set(dr_mst["medicine code"].unique().tolist())
yj7_codes = set(dr_mst["yj_7"].unique().tolist())

In [18]:
dr_mst.head()

,medicine code,brand name (in Japanese),generic name,Class,yj,yj_7
0,622740600,アカルボース５０ｍｇ錠,Acarbose,α-glucosidase inhibitor,3969003F1018,3969003
1,613960081,グルコバイ錠５０ｍｇ,Acarbose,α-glucosidase inhibitor,3969003F1026,3969003
2,620005359,アカルボース錠５０ｍｇ「タイヨー」,Acarbose,α-glucosidase inhibitor,3969003F1034,3969003
3,620009291,アカルボース錠５０ｍｇ「ＢＭＤ」,Acarbose,α-glucosidase inhibitor,3969003F1042,3969003
4,620009289,アカルボース錠５０ｍｇ「ＪＧ」,Acarbose,α-glucosidase inhibitor,3969003F1050,3969003


In [19]:
print(d_mst.shape)
d_mst.head()

(309, 4)


,diagnosis in Japanese,diagnosis in English,ICD-10 code,diagnosis code
0,インスリン抵抗性糖尿病,Insulin resistant diabetes mellitus,E11,2500001
1,糖尿病,Diabetes mellitus,E14,2500013
2,１型糖尿病,Type 1 diabetes mellitus,E10,2500014
3,２型糖尿病,Type 2 diabetes mellitus,E11,2500015
4,膵性糖尿病,Pancreatic diabetes mellitus,E13,2500024


In [20]:
f = "../../share3/m_icd10.parquet"
dis_mst = pl.scan_parquet(f)\
    .filter(
        (pl.col("icd10_code").is_in(dm_icd10)) | (pl.col("diseases_code").is_in(dm_codes))
    )
dis_mst.collect()

icdset = dis_mst.collect()\
    .select(pl.col("icd10_code")).to_series().to_list()
icdset = set(icdset)
disset = dis_mst.collect()\
    .select(pl.col("diseases_code")).to_series().to_list()
disset = set(disset)
len(disset)

306

In [21]:
# disset
f_ = "../../share3/mat/dm_codes.json"
d2j(d={"dmcode":list(disset)}, f=f_)

In [22]:
f = "../../share3/m_drug_who_atc.parquet"
m_drg_atc = pl.scan_parquet(f)\
    .filter(
        pl.col("atc_medium_code") == "A10"    
        # (pl.col("drug_code").is_in(dr_codes))
    )\
    .select(
        pl.col("drug_code", "atc_code", "atc_sub_code", "atc_sub_name", "atc_subdiv_code", "atc_subdiv_name")
    )
drg_codes1 = m_drg_atc.collect()\
    .select(pl.col("drug_code")).to_series().to_list()

In [23]:
# dr_codes
atcsb = m_drg_atc\
    .select(pl.col("atc_subdiv_code")).collect().to_series().unique().to_list()

m_drg_atc.collect()

drug_code,atc_code,atc_sub_code,atc_sub_name,atc_subdiv_code,atc_subdiv_name
str,str,str,str,str,str
"""620000265""","""A10AB01""","""A10A""","""インスリンおよびその誘導体""","""A10AB""","""注射用インスリンおよびその誘…"
"""620008897""","""A10AB01""","""A10A""","""インスリンおよびその誘導体""","""A10AB""","""注射用インスリンおよびその誘…"
"""622114401""","""A10AB01""","""A10A""","""インスリンおよびその誘導体""","""A10AB""","""注射用インスリンおよびその誘…"
"""620008909""","""A10AB01""","""A10A""","""インスリンおよびその誘導体""","""A10AB""","""注射用インスリンおよびその誘…"
"""620008907""","""A10AB01""","""A10A""","""インスリンおよびその誘導体""","""A10AB""","""注射用インスリンおよびその誘…"
…,…,…,…,…,…
"""621682502""","""A10XA""","""A10X""","""その他の糖尿病用薬""","""A10XA""","""アルドース還元酵素阻害薬"""
"""621679001""","""A10XA""","""A10X""","""その他の糖尿病用薬""","""A10XA""","""アルドース還元酵素阻害薬"""
"""622088001""","""A10XA""","""A10X""","""その他の糖尿病用薬""","""A10XA""","""アルドース還元酵素阻害薬"""


In [24]:
# atcsb
f_atcsb = "../../share3/mat/atc_codes.json"
d2j(d={"atcsubcode":atcsb}, f=f_atcsb)

In [25]:
f = "../../share3/m_drug_main.parquet"
m_drg = pl.scan_parquet(f)\
    .with_columns(
        pl.col("yj_code").str.slice(0,7).alias("yj_7")
    )\
    .filter(
        (pl.col("drug_code").is_in(dr_codes))|
        (pl.col("yj_7").is_in(yj7_codes))
    )
m_drg.collect()
drg_codes2 = m_drg.collect()\
    .select(pl.col("drug_code")).to_series().to_list()

drg_codes = set(drg_codes1 + drg_codes2)
# drg_codes

In [26]:
# drugcode
f_ = "../../share3/mat/drug_codes.json"
l_drg_codes = list(drg_codes)
d2j(d={"drugcode":l_drg_codes}, f=f_)

In [33]:
l_cancer = ["C" + str(i).zfill(2) for i in range(0, 100)] + \
        ["D" + str(i).zfill(2) for i in range(0, 9)] + \
        ["D" + str(i).zfill(2) for i in range(37, 49)]
tag_cancer = set(l_cancer)

In [34]:
f_dcans = "../../share3/mat/cans_codes_icd.json"
d2j(d={"icdcode":l_cancer}, f=f_dcans)



In [35]:
f = "../../share3/m_icd10.parquet"
m_dis = pl.scan_parquet(f)\
    .filter(
        pl.col("icd10_sub_code").is_in(tag_cancer)
    )\
    .select(
        pl.col("diseases_code", "icd10_kbn_code", "icd10_code","icd10_sub_code")
    )
code_cancer = m_dis\
    .collect()\
    .select(
            pl.col("diseases_code")
    ).to_series()\
    .unique().to_list()

f_dcans = "../../share3/mat/cans_codes.json"
d2j(d={"cancercode":code_cancer}, f=f_dcans)

code_cancer = set(code_cancer )
m_dis.fetch()

diseases_code,icd10_kbn_code,icd10_code,icd10_sub_code
str,str,str,str
"""8836431""","""1""","""C01""","""C01"""
"""8849995""","""1""","""C07""","""C07"""
"""8849994""","""1""","""C07""","""C07"""
"""8849993""","""1""","""C07""","""C07"""
"""1420003""","""1""","""C07""","""C07"""
…,…,…,…
"""1939005""","""1""","""C73""","""C73"""
"""1939009""","""1""","""C73""","""C73"""
"""8849964""","""1""","""C73""","""C73"""


In [10]:
# disset

In [27]:

# dis_mst.fetch()
d_mst\
    .query("`diagnosis code` not in @disset and `ICD-10 code` not in @icdset")
    # .query("`ICD-10 code` not in @icdset")



,diagnosis in Japanese,diagnosis in English,ICD-10 code,diagnosis code


In [12]:
lx = [str(i) for i in range(10)]
lx

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

In [21]:
%%time
for i in lx:
    f = "../../share3/dis/df_dis_split_{}.parquet".format(i)
    dis_ = pl.scan_parquet(f)\
        .filter(pl.col("utagai_flg") != "1")\
        .filter(pl.col("diseases_code").is_in(disset))\
        .group_by("kojin_id","grp", "receipt_ym","receipt_id")\
        .agg(pl.lit(True).alias("with_dm_diag_codes"))

    f = "../../share3/drg/df_drg_split_{}.parquet".format(i)
    drg_ = pl.scan_parquet(f)\
        .filter(pl.col("drug_code").is_in(dr_codes))\
        .join(other=m_drg_atc.lazy(), how="left", on="drug_code")\
        .group_by("kojin_id","grp", "receipt_ym","receipt_id", "atc_subdiv_code")\
        .agg(pl.col("line_no").count().alias("dm_drg_usage_sub"))\
        .group_by("kojin_id","grp", "receipt_ym","receipt_id")\
        .agg(
            (pl.col("atc_subdiv_code") == value).max().alias(value)
            for value in atcsb
        )

    f_out = "../../share3/diag/df_dm_diag_split_{}.parquet".format(i)
    print(f_out)
    dm_diag = drg_\
        .join(other=dis_,
              how ="inner",
              on=["kojin_id","grp", "receipt_ym","receipt_id"])\
        .group_by("kojin_id", "grp", "receipt_ym")\
        .agg(
            pl.col(value).max().alias(value)
            for value in atcsb + ["with_dm_diag_codes"]
        )
    
    # dm_diag.sink_parquet(f_out)
    dm_diag.collect(streaming=True).write_parquet(f_out)
    gc.collect()
    # break

../../share3/diag/df_dm_diag_split_0.parquet
../../share3/diag/df_dm_diag_split_1.parquet
../../share3/diag/df_dm_diag_split_2.parquet
../../share3/diag/df_dm_diag_split_3.parquet
../../share3/diag/df_dm_diag_split_4.parquet
../../share3/diag/df_dm_diag_split_5.parquet
../../share3/diag/df_dm_diag_split_6.parquet
../../share3/diag/df_dm_diag_split_7.parquet
../../share3/diag/df_dm_diag_split_8.parquet
../../share3/diag/df_dm_diag_split_9.parquet
CPU times: user 27min 29s, sys: 2min 8s, total: 29min 37s
Wall time: 4min 25s


In [17]:
# dm_diag.show_graph(streaming=True)